In [1]:
pip install --upgrade mit-news-classify

     |████████████████████████████████| 2.1MB 13.8MB/s 
     |████████████████████████████████| 24.2MB 71.4MB/s 
     |████████████████████████████████| 3.3MB 64.7MB/s 
     |████████████████████████████████| 901kB 63.1MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
%%time
from mitnewsclassify import download

download.download()

model_2500_500_50.h5?dl=1: 0.00B [00:00, ?B/s]Package directory: /home/mbaliesnyi/anaconda3/envs/nlp/lib/python3.8/site-packages/mitnewsclassify
/home/mbaliesnyi/anaconda3/envs/nlp/lib/python3.8/site-packages/mitnewsclassify/data directory already exists, some other models downloaded. Continuing...
/home/mbaliesnyi/anaconda3/envs/nlp/lib/python3.8/site-packages/mitnewsclassify/data/tfidf directory already exists... perhaps you already downloaded the data? Overwriting...
/home/mbaliesnyi/anaconda3/envs/nlp/lib/python3.8/site-packages/mitnewsclassify/data/tfidf_bi directory already exists... perhaps you already downloaded the data? Overwriting...
/home/mbaliesnyi/anaconda3/envs/nlp/lib/python3.8/site-packages/mitnewsclassify/data/doc2vec directory already exists... perhaps you already downloaded the data? Overwriting...
/home/mbaliesnyi/anaconda3/envs/nlp/lib/python3.8/site-packages/mitnewsclassify/data/gpt2 directory already exists... perhaps you already downloaded the data? Overwriting

In [3]:
# download NYT dataset
!curl https://www.dropbox.com/sh/xu9tu5hmjhuddwk/AACFtsyeBeB7mw7WW37935cYa/NYTcorpus_test.p.gz -LO
!curl https://www.dropbox.com/sh/xu9tu5hmjhuddwk/AAD31tK6oEoGlhpRZzeu3Y3Ya/NYTcorpus_train.p.gz -LO

# train and test data labels are coded in numbers,
# but the models predict human-readable labels,
# so we need to map these for model evaluation. 
# Let's use one of the files downloaded by the mitnewsclassify package
!curl https://www.dropbox.com/s/omgstbndd3xl4cy/nyt-theme-tags.csv -LO

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  1076    0  1076    0     0   1016      0 --:--:--  0:00:01 --:--:--  1016
100  210M  100  210M    0     0  76.2M      0  0:00:02  0:00:02 --:--:--  153M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  1076    0  1076    0     0    849      0 --:--:--  0:00:01 --:--:--   849
100 1899M  100 1899M    0     0  96.7M      0  0:00:19  0:00:19 --:--:--  126M
  % Total    % Received % Xferd  Average Speed   Tim

In [5]:
import gzip
import pickle
from random import shuffle
import requests
import csv

# open the train data given to us by Max
with gzip.open('NYTcorpus_train.p.gz', mode='r') as f:
    train_data = pickle.load(f)

# open the test data given to us by Max
with gzip.open('NYTcorpus_test.p.gz', mode='r') as f:
    test_data = pickle.load(f)

# shuffle just in case the test and train data were not shuffled before - 
# we will only measure model's accuracy on a few thousand samples
shuffle(train_data)
shuffle(test_data)

# train and test data labels are coded in numbers,
# but the models predict human-readable labels,
# so we need to re-map these. 
# Let's use one of the files downloaded by the mitnewsclassify package
with open('nyt-theme-tags.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    tags_dict = {row['tags_id']: row['tag'] for row in reader}


# extract actual article texts from data samples
train_articles = [d[2] for d in train_data] 
test_articles = [d[2] for d in test_data]

# map the number-coded labels to human-readable labels
train_labels_lists = [list(map(tags_dict.get, d[3:])) for d in train_data]
test_labels_lists = [list(map(tags_dict.get, d[3:])) for d in test_data]

In [6]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

def multi_label_scores(correct_labels, predicted_labels):

    accuracy = accuracy_score(correct_labels, predicted_labels)
    precision = precision_score(correct_labels, predicted_labels, average='weighted', zero_division=0)
    recall = recall_score(correct_labels, predicted_labels, average='weighted', zero_division=0)
    f_1_score = f1_score(correct_labels, predicted_labels, average='weighted', zero_division=0)
    
    return accuracy, precision, recall, f_1_score


In [7]:
from sklearn.preprocessing import MultiLabelBinarizer

# get the set of all 538 labels for fitting MultiLabelBinarizer
labels_flattened = []
for label in test_labels_lists:
    labels_flattened.extend(label)

label_names = list(set(labels_flattened))

mlb = MultiLabelBinarizer()
mlb.fit([sorted(label_names)])


MultiLabelBinarizer(classes=None, sparse_output=False)

In [8]:
import warnings
from tqdm.autonotebook import tqdm

warnings.simplefilter(action='ignore', category=FutureWarning)


def evaluate(model, articles, labels_lists, n_of_articles = 100):

    model_name = model.__name__.split('.')[1]

    articles_subset = articles[:n_of_articles]
    labels_subset = labels_lists[:n_of_articles]

    print('predicting for', model_name)

    preds, correct = [], []

    for article, labels in tqdm(zip(articles_subset, labels_subset)):
        pred_labels = model.gettags(article)
        preds.append(pred_labels)
        correct.append(labels)

    correct, preds = mlb.transform(correct), mlb.transform(preds)
    accuracy, precision, recall, f_score = multi_label_scores(correct, preds)
    print(f'accuracy {round(accuracy,4)}, precision {round(precision,4)}, recall {round(recall,4)}, f-1 {round(f_score,4)}')
    print()
    return model_name, dict(accuracy=accuracy, precision=precision, recall=recall, f_score = f_score)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  


In [10]:
from mitnewsclassify import download
download.download()

model_2500_500_50.h5?dl=1: 0.00B [00:00, ?B/s]

Package directory: /usr/local/lib/python3.7/dist-packages/mitnewsclassify


model_2500_500_50.h5?dl=1: 100%|██████████| 121M/121M [00:02<00:00, 45.2MB/s]
small_vocab_20.csv?dl=1: 0.00B [00:00, ?B/s]

small_vocab_20.csv?dl=1: 100%|██████████| 32.8k/32.8k [00:01<00:00, 30.1kB/s]
tfmer_20.p?dl=1: 0.00B [00:00, ?B/s]

tfmer_20.p?dl=1: 100%|██████████| 57.3k/57.3k [00:00<00:00, 58.5kB/s]
labelsdict_20.p?dl=1: 0.00B [00:00, ?B/s]

...

labelsdict.p?dl=1: 100%|██████████| 16.4k/16.4k [00:00<00:00, 19.1kB/s]
nyt-theme-tags.csv?dl=1: 0.00B [00:00, ?B/s]

nyt-theme-tags.csv?dl=1: 100%|██████████| 24.6k/24.6k [00:00<00:00, 26.6kB/s]
model_pentasemble.h5?dl=1: 0.00B [00:00, ?B/s]

model_pentasemble.h5?dl=1: 100%|██████████| 114M/114M [00:01<00:00, 60.2MB/s]
labelsdict.p?dl=1: 0.00B [00:00, ?B/s]

labelsdict.p?dl=1: 100%|██████████| 16.4k/16.4k [00:00<00:00, 19.3kB/s]
nyt-theme-tags.csv?dl=1: 0.00B [00:00, ?B/s]

nyt-theme-tags.csv?dl=1: 100%|██████████| 24.6k/24.6k [00:01<00:00, 19.5kB/s]


In [11]:
from mitnewsclassify import gpt2
from mitnewsclassify import tfidf
from mitnewsclassify import tfidf_bi
from mitnewsclassify import doc2vec
from mitnewsclassify import ensemble
from mitnewsclassify import trisemble
from mitnewsclassify import quadsemble
from mitnewsclassify import pentasemble

models = [
    gpt2,
    tfidf_bi,
    doc2vec,
    quadsemble,
    pentasemble,
    ensemble,
    trisemble,
    tfidf,
]

train_set_scores = {}
test_set_scores = {}

for model in models:
    print('Train: ')
    model_name, model_scores = evaluate(model, train_articles, train_labels_lists, n_of_articles=500)
    train_set_scores[model_name] = model_scores
    
    print('Test: ')
    model_name, model_scores = evaluate(model, test_articles, test_labels_lists, n_of_articles=500)
    test_set_scores[model_name] = model_scores


Train: 
predicting for gpt2


WARNING This model will consume a lot of memory, which can render your computer unusable. Please make sure that you have sufficient memory!


Some weights of GPT2Model were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Initializing...
Model...
Miscellaneous...

accuracy 0.0, precision 0.0398, recall 0.035, f-1 0.0293

Test: 
predicting for gpt2



accuracy 0.0, precision 0.0456, recall 0.0269, f-1 0.0232

Train: 
predicting for tfidf_bi


...


accuracy 0.4, precision 0.7551, recall 0.6102, f-1 0.6573

Train: 
predicting for trisemble


WARNING This model will consume a lot of memory, which can render your computer unusable. Please make sure that you have sufficient memory!
Initializing...
Model...
Miscellaneous...

accuracy 0.408, precision 0.8117, recall 0.6965, f-1 0.7335

Test: 
predicting for trisemble



accuracy 0.398, precision 0.7685, recall 0.6155, f-1 0.6651

Train: 
predicting for tfidf



accuracy 0.39, precision 0.7888, recall 0.6446, f-1 0.6926

Test: 
predicting for tfidf



accuracy 0.398, precision 0.7562, recall 0.601, f-1 0.6519



In [12]:
all_train_results = sorted(train_set_scores.items(), key=lambda k: k[1]['f_score'])

all_test_results = sorted(test_set_scores.items(), key=lambda k: k[1]['f_score'])

# best model
all_train_results[-1], all_test_results[-1]

(('quadsemble',
  {'accuracy': 0.426,
   'f_score': 0.7465092459029774,
   'precision': 0.8098629290154356,
   'recall': 0.7178988326848249}),
 ('quadsemble',
  {'accuracy': 0.386,
   'f_score': 0.6832388013752136,
   'precision': 0.7748118603208904,
   'recall': 0.639763779527559}))

In [13]:
from pprint import pprint
art = train_articles[10]
pprint(art)


('AMERICANS are falling out of love with cars. In a speech last week, none '
 'other than William Clay Ford Jr., the chairman and chief executive of the '
 "Ford Motor Company, proclaimed the end of the affair. ''The love affair that "
 "people have had with automobiles has in some ways grown stale,'' he said. "
 "''Some would say it's even dying.'' Mr. Ford chalked it up to the long "
 'running conflict between carmakers and regulators, particularly in '
 "California, where ''people used to write songs about T-Birds and "
 "Corvettes.'' ''Today,'' he said, ''they write regulations.'' From his "
 'perspective, it will take continued effort on breakthrough environmental '
 'technologies and a more cooperative approach with regulators to turn things '
 'around. Then, he said, people might not be so offended by what their '
 'neighbors are driving. This sentiment cheered environmental activists, who '
 "think the growth of S.U.V.'s, like those Ford produces, has shifted "
 'automobiles fr

In [14]:
gpt2.gettags(art)

['consumer protection', 'legal profession']

In [16]:
quadsemble.gettags(art)

['environment', 'automobiles']